# Notebook 3: Retrieve Echotop Field from Pre-Computed Data

In Notebook 2, we pre-computed the Echotop variable from raw model data and stored it back into FDB. Now, in Notebook 3, our objective is to retrieve this pre-computed echo-top field. This field, initially not generated as part of the model's output, is highly valuable for numerous downstream applications, making it of considerable interest for many users to retrieve. Additionally, we apply a regridding of the data to the Swiss grid coordinate system (EPSG code 21781) to showcase another feature. By accessing this stored data, we streamline the process of accessing and utilizing essential meteorological information, enabling smoother workflows and improved insights.

1. **Configuring Access to Polytope**

In [ ]:
import os

os.environ["POLYTOPE_ADDRESS"] = "https://polytope-dev.mchml.cscs.ch"
#os.environ["POLYTOPE_USERNAME"] = "admin"
#os.environ["POLYTOPE_PASSWORD"] = "xxxxxxxxxxxxx"


2. **Querying Data**

In [ ]:
from meteodatalab import mars, mch_model_data

request = mars.Request(
    param="ECHOTOPinM",
    date="20240409",
    time="1800",
    number=0,
    step=60,
    levtype=mars.LevType.SURFACE,
    model=mars.Model.ICON_CH1_EPS,
)

ds = mch_model_data.get_from_polytope(request)


3. **Regridding to the Swiss grid coordinate system**

In [ ]:
from meteodatalab.operators.regrid import regrid, RegularGrid, Resampling, CRS

echo_top = ds["ECHOTOPinM"]
dst = RegularGrid(
    crs=CRS.from_epsg(21781),
    xmin=400000,
    xmax=900000,
    ymin=50000,
    ymax=350000,
    nx=500,
    ny=300,
)
echo_top_lv03 = regrid(echo_top, dst, Resampling.bilinear)


4. **Visualizing the data**

In [ ]:
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from cartopy import feature as cfeature

crs = ccrs.epsg(21781)
fig = plt.figure(figsize=(11, 8))
ax = fig.add_subplot(1, 1, 1, projection=crs)

f = echo_top_lv03.squeeze()
c = ax.contourf(dst.x, dst.y, f.values, transform=crs)
fig.colorbar(c, label="m")

ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)

ax.set_title("Echo top in meters")
